## **1. Introduction et contexte physique**

Les équations différentielles (ED) sont utilisées pour **modéliser des phénomènes physiques**, comme le mouvement des corps, les circuits électriques ou les systèmes mécaniques.

Exemple classique : le mouvement d’un point matériel :

$$
m \vec{x}''(t) = \vec{F}(t)
$$

### Forces courantes et leurs ED

| Type de force                           | Équation différentielle                                  |
| --------------------------------------- | -------------------------------------------------------- |
| Force externe seule                     | $$ m\vec{x}'' = \vec{F}(t)$$                               |
| + frottement proportionnel à la vitesse | $$m\vec{x}'' + \gamma \vec{x}' = \vec{F}(t) $$            |
| + ressort                               | $$ m\vec{x}'' + \gamma \vec{x}' + k\vec{x} = \vec{F}(t) $$ |

**Remarque :** tant que les forces dépendent **linéairement** de (\vec{x}) et (\vec{x}'), l’équation reste linéaire.

### **Exercice type**

**Énoncé :**
Un corps de masse (m=1) est soumis à un frottement $$\gamma=2$$ et à une force externe $$F(t)=\cos(t)). Résoudre (x''+2x' = \cos(t)), (x(0)=0, x'(0)=0$$.

**Python solution :**


In [ ]:
import sympy as sp

t = sp.symbols('t')
x = sp.Function('x')(t)
ode = sp.Eq(x.diff(t,2) + 2*x.diff(t), sp.cos(t))
sol = sp.dsolve(ode, ics={x.subs(t,0):0, x.diff(t).subs(t,0):0})
print(sol)

## **2. Équations différentielles linéaires du premier ordre**

### Définition

$$
x'(t) = P(t)x(t) + Q(t), \quad P, Q \text{ continues}
$$

### Solution analytique

* **Cas homogène** ((Q=0)) :
  $$
  x(t) = x_0 \exp\Big(\int_{t_0}^{t} P(s) ds\Big)
  $$

* **Cas non homogène** ((Q\neq0), Duhamel) :
  $$
  x(t) = x_0 \exp\Big(\int_{t_0}^{t} P(s) ds\Big) + \int_{t_0}^{t} \exp\Big(\int_s^t P(u) du\Big) Q(s) ds
  $$

### Exemple type

$$
x'(t) = x(t) + t, \quad x(0)=1
$$

**Solution analytique** :

$$
x(t) = e^t (x_0 -1) - t + e^t
$$

**Python :**


In [ ]:
import sympy as sp
t = sp.symbols('t')
x0 = 1
x = sp.Function('x')(t)
ode = sp.Eq(x.diff(t), x + t)
sol = sp.dsolve(ode, ics={x.subs(t,0): x0})
print(sol)

### **Exercice type**

Résoudre (x'(t) = 2x(t) + e^t), (x(0)=0).

---

## **3. Systèmes linéaires du premier ordre**

### Définition

$$
X'(t) = A(t)X(t) + Q(t), \quad X \in \mathbb{R}^n
$$

* Si (A) constant :
  $$
  X(t) = e^{A(t-t_0)} X(t_0) + \int_{t_0}^t e^{A(t-s)} Q(s) ds
  $$

* **Exponentielle de matrice** :
  $$
  e^A = \sum_{n=0}^{\infty} \frac{A^n}{n!}
  $$

### Exemple 2×2

$$
\begin{cases}
x_1' = x_1 + x_2 + t\
x_2' = x_1 - x_2 + t
\end{cases}, \quad
A = \begin{pmatrix}1 & 1 \ 1 & -1\end{pmatrix}
$$

**Python solution :**


In [ ]:
import numpy as np
from scipy.linalg import expm
from scipy.integrate import quad

A = np.array([[1.,1.],[1.,-1.]])
X0 = np.zeros(2)
t_eval = 0.5

def integrand(s, comp):
    M = expm(A*(t_eval-s))
    return M[comp,:].dot(np.array([s,s]))

I0 = quad(lambda s: integrand(s,0),0,t_eval)[0]
I1 = quad(lambda s: integrand(s,1),0,t_eval)[0]
X_t = expm(A*t_eval).dot(X0) + np.array([I0,I1])
print("X(0.5) =", X_t)


## **4. Méthodes numériques pour ODEs**

### Schémas principaux

| Schéma          | Formule                                                                                  | Commentaire                               |
| --------------- | ---------------------------------------------------------------------------------------- | ----------------------------------------- |
| Euler explicite | $$x_{n+1} = x_n + \Delta t f(t_n,x_n)$$                                                    | simple, instable si (\Delta t) trop grand |
| Euler implicite | $$x_{n+1} = x_n + \Delta t f(t_{n+1}, x_{n+1})$$                                       | stable pour tout (\Delta t)               |
| Heun            | $$x_{n+1} = x_n + \frac{\Delta t}{2}[f(t_n,x_n) + f(t_{n+1}, x_n + \Delta t f(t_n,x_n))]$$ | précision d’ordre 2                       |

### Exemple type : (x'=-5x, x_0=1, T=1)

**Python :**


In [ ]:
import numpy as np

gamma = 5.0
x0 = 1.0
T = 1.0
N = 100
dt = T/N

def euler_exp(f,x0):
    x = x0
    for _ in range(N):
        x = x + dt*f(x)
    return x

f = lambda x: -gamma*x
x_euler = euler_exp(f, x0)
print(x_euler)

## **5. Problèmes aux limites (2ᵉ ordre)**

### Forme générale

$$
-(\alpha u')' + \beta u' + \gamma u = f(x), \quad x \in [a,b]
$$

* **Conditions aux bords :** Dirichlet, Neumann, Robin
* **Solution générale :**
  $$
  u(x) = c_1 u_1(x) + c_2 u_2(x) + u_p(x)
  $$

### Exemple type

$$
u''(x)=1, \quad u(0)=0, u'(1)=0 \Rightarrow u(x) = \frac{x^2}{2}-x
$$

---

## **6. Discrétisation 1D par différences finies**

$$
x_i = i h, \quad h = \frac{1}{N+1}
$$

$$
\frac{-u_{i-1}+2u_i-u_{i+1}}{h^2} = f_i, \quad u_0=u_{N+1}=0
$$

**Python :**



In [ ]:
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import numpy as np

N = 100
h = 1/(N+1)
x = np.linspace(h,1-h,N)
F = np.sin(np.pi*x)
A = diags([-1,2,-1],[-1,0,1],shape=(N,N))/h**2
U = spsolve(A,F)

# **Exercice 1 — ODE du premier ordre (analytique + vérification)**

**Énoncé :**
Résoudre
[
x'(t) = x(t) + t, \quad x(0)=1
]

1. Donner la solution analytique.
2. Vérifier numériquement la solution et comparer l’erreur.

**Python :**

```python
import numpy as np
import sympy as sp
from scipy.integrate import solve_ivp

# Analytique avec sympy
t = sp.symbols('t', real=True)
x0 = 1
x = sp.Function('x')(t)
ode = sp.Eq(x.diff(t), x + t)
sol = sp.dsolve(ode, ics={x.subs(t,0): x0})
print("Solution analytique :", sp.simplify(sol.rhs))

# Vérification numérique
def f(t, y): return y + t
ts = np.linspace(0, 2, 201)
sol_num = solve_ivp(f, [0, 2], [x0], t_eval=ts, rtol=1e-9, atol=1e-12)
x_exact = lambda T: x0*np.exp(T) - T + np.exp(T) - 1  # formule analytique
err = np.max(np.abs(sol_num.y[0] - x_exact(ts)))
print("Erreur max :", err)
```

**Explication :**

* `sympy.dsolve` résout symboliquement l’ODE.
* `solve_ivp` permet de vérifier la solution numériquement.

---

# **Exercice 2 — Système linéaire 2×2**

**Énoncé :**
Résoudre le système
[
X'(t) = A X(t) + \begin{pmatrix} t \ t \end{pmatrix}, \quad X(0) = (0,0)^T
]
avec
[
A = \begin{pmatrix} 1 & 1 \ 1 & -1 \end{pmatrix}
]
Évaluer (X(0.5)).

**Python :**

```python
import numpy as np
from scipy.linalg import expm
from scipy.integrate import quad

A = np.array([[1.,1.],[1.,-1.]])
X0 = np.zeros(2)
t_eval = 0.5

def integrand(s, comp):
    M = expm(A*(t_eval-s))
    return M[comp,:].dot(np.array([s,s]))

I0 = quad(lambda s: integrand(s,0), 0, t_eval, epsabs=1e-9)[0]
I1 = quad(lambda s: integrand(s,1), 0, t_eval, epsabs=1e-9)[0]
X_t = expm(A*t_eval).dot(X0) + np.array([I0, I1])
print("X(0.5) =", X_t)
```

**Explication :**

* `expm` calcule l’exponentielle de matrice (e^{At}).
* Intégration numérique avec `quad` pour la convolution avec (Q(t)).

---

# **Exercice 3 — Comparer Euler explicite et Heun**

**Énoncé :**
Pour (x' = x, x(0) = 1), comparer les schémas Euler explicite et Heun à (T=1) pour (N \in {10,50,200,1000}).

**Python :**

```python
import numpy as np
import math

def euler(f, t0, x0, T, N):
    dt = (T-t0)/N
    x = x0
    t = t0
    for _ in range(N):
        x = x + dt * f(t, x)
        t += dt
    return x

def heun(f, t0, x0, T, N):
    dt = (T-t0)/N
    x = x0
    t = t0
    for _ in range(N):
        k1 = f(t, x)
        k2 = f(t+dt, x + dt*k1)
        x = x + dt*(k1 + k2)/2
        t += dt
    return x

f = lambda t,x: x
T = 1.0
x_true = math.exp(1.0)
for N in [10,50,200,1000]:
    xe = euler(f,0.0,1.0,T,N)
    xh = heun(f,0.0,1.0,T,N)
    print(N, "Euler err", abs(xe-x_true), "Heun err", abs(xh-x_true))
```

**Explication :**

* Euler explicite simple mais moins précis.
* Heun améliore l’ordre de convergence (ordre 2).

---

# **Exercice 4 — Stabilité**

**Énoncé :**
Pour (x'=-100x, x_0=1), comparer Euler explicite, Euler implicite et Heun pour (\Delta t \in {0.005,0.02,0.05,0.1}) jusqu’à (T=1).

**Python :**

```python
import numpy as np

gamma = 100.0
f = lambda t,x: -gamma*x
T = 1.0
x0 = 1.0

def euler_explicit_step(x, dt): return x*(1 - gamma*dt)
def euler_implicit_step(x, dt): return x/(1 + gamma*dt)
def heun_step(x, dt):
    k1 = -gamma*x
    k2 = -gamma*(x + dt*k1)
    return x + dt*(k1 + k2)/2

for dt in [0.005,0.02,0.05,0.1]:
    N = int(T/dt)
    x = x0
    for _ in range(N): x = euler_explicit_step(x, dt)
    x_e = x
    x = x0
    for _ in range(N): x = euler_implicit_step(x, dt)
    x_i = x
    x = x0
    for _ in range(N): x = heun_step(x, dt)
    x_h = x
    print(f"dt={dt:.3f} -> Euler exp {x_e:.3e}, Euler imp {x_i:.3e}, Heun {x_h:.3e}")
```

**Explication :**

* Euler explicite diverge pour gros (\Delta t).
* Euler implicite stable pour tous (\Delta t).
* Heun semi-stable, meilleure précision.

---

# **Exercice 5 — Poisson 1D avec différences finies**

**Énoncé :**
Résoudre (-u''(x) = f(x), x\in(0,1), u(0)=u(1)=0), avec (f(x)=\sin(\pi x)). Comparer solution numérique et analytique (u(x)=\sin(\pi x)/\pi^2).

**Python :**

```python
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import numpy as np

N = 200
h = 1.0/(N+1)
x = np.linspace(h,1-h,N)
F = np.sin(np.pi*x)
A = diags([-1,2,-1],[-1,0,1],shape=(N,N),format='csr')/h**2
U = spsolve(A,F)
u_exact = np.sin(np.pi*x)/(np.pi**2)
print("Erreur max:", np.max(np.abs(U-u_exact)))
```

**Explication :**

* Discrétisation par différences finies → matrice tridiagonale creuse.
* Résolution rapide avec `spsolve`.

---

# **Exercice 6 — Calcul de (e^A) pour matrice 3×3**

**Énoncé :**
Générer une matrice aléatoire (A \in \mathbb{R}^{3×3}). Si diagonalisable (A=SΛS^{-1}), vérifier (e^{At} ≈ S e^{Λ t} S^{-1}). Sinon utiliser `scipy.linalg.expm`.

**Python :**

```python
import numpy as np
from scipy.linalg import expm, eig

np.random.seed(0)
A = np.random.randn(3,3)
t = 1.0

# diagonalisation
w, S = eig(A)
if np.linalg.matrix_rank(S) == 3:
    D = np.diag(w)
    exp_via_diag = S.dot(np.diag(np.exp(w*t))).dot(np.linalg.inv(S))
    exp_direct = expm(A*t)
    print("Max diff diag vs expm:", np.max(np.abs(exp_via_diag - exp_direct)))
else:
    print("Pas diagonalisable, utiliser expm")
    print(expm(A*t))
```
